# Code 4-52 to Code 4-63

In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
pd.options.display.max_colwidth = 1000

We are reading the initial NER dataset 
we would then split the sentences to words and the corresponding  POS tags like we did for the training set in code "4_NER_News_new". 

In [2]:
t1 = pd.read_csv("news_summary.csv",encoding="latin1")
t1["imp_col"] = t1["headlines"] + " " + t1["text"] + " " + t1["ctext"]

t2 = t1[t1.imp_col.str.len()>=10].reset_index()
len(t2),len(t1)

t2.shape, t2.columns


((4396, 8),
 Index(['index', 'author', 'date', 'headlines', 'read_more', 'text', 'ctext',
        'imp_col'],
       dtype='object'))

# Preparing the data set for the model
step 1 : convert to POS Tags


In [3]:

lemmatizer = WordNetLemmatizer() 
for num,i in enumerate(t2.loc[:,"imp_col"]):
    
    for num1,j in enumerate(nltk.sent_tokenize(i)):
        pos_tags_list = nltk.pos_tag(nltk.word_tokenize(j))
        df = pd.DataFrame(pos_tags_list)
        df.columns = ["word","pos_tag"]
        df["sid"] = num1
        df["sid1"] = num
        if(num==0):
            df_all = df
        else:
            df_all = pd.concat([df_all,df],axis=0)
def lemma_func(x):
    return lemmatizer.lemmatize(x)

df_all_pos = df_all



In [4]:
len(df_all)

2077452

Step2: To create word type variables, lemma of words and the relative length variables

In [5]:
df_all_pos["word1"] = df_all_pos["word"] 

df_all_pos["word1"] = df_all_pos.word1.str.replace('[^a-z\s]+','') 

df_all_pos["word_type"] = "normal"
df_all_pos.loc[df_all_pos.word.str.contains('[0-9]+'),"word_type"] = "number"
df_all_pos.loc[df_all_pos.word.str.contains('[^a-zA-Z\s]+'),"word_type"] = "special_chars"

df_all_pos.loc[(df_all_pos.word_type!="normal") & (df_all_pos.word1.str.len()==0),"word_type"] = "only_special"

In [6]:
df_all_pos["shape"] = "mixed"
df_all_pos.loc[((df_all_pos.word.str.islower()==True) & (df_all_pos.word_type=="normal")),"shape"]="lower"
df_all_pos.loc[((df_all_pos.word.str.islower()==False) & (df_all_pos.word_type=="normal")),"shape"]="upper"


df_all_pos["lemma"] = df_all_pos["word"].apply(lemma_func)

df_all_pos["length"] = df_all_pos["word"].str.len()

In [7]:
df_all_pos["ind_num"] = df_all_pos.index
df_all_pos["sent_len"]=df_all_pos.groupby(["sid","sid1"])["ind_num"].transform(max)
df_all_pos1 = df_all_pos[df_all_pos.sent_len>0]
df_all_pos1["rel_position"] = df_all_pos1["ind_num"] /df_all_pos1["sent_len"]*100
df_all_pos1["rel_position"] = df_all_pos1["rel_position"].astype('int')

C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Step 3: We now get the positional variables like previous, next etc

In [8]:
def get_prev_next(df_all_pos,col_imp):
    prev_col = "prev_" + col_imp
    next_col = col_imp + "_next"
    
    prev_col1 = "prev_prev_" + col_imp
    next_col1 = col_imp + "_next_next"
    
    df_all_pos[prev_col] = df_all_pos[col_imp].shift(1)
    df_all_pos.loc[df_all_pos.index==0,prev_col] = "start"
    
    df_all_pos[next_col] = df_all_pos[col_imp].shift(-1)
    df_all_pos.loc[df_all_pos.index==df_all_pos.sent_len,next_col] = "end"
    
    df_all_pos[prev_col1] = df_all_pos[col_imp].shift(2)
    df_all_pos.loc[df_all_pos.index<2,prev_col1] = "start"
    
    df_all_pos[next_col1] = df_all_pos[col_imp].shift(-2)
    df_all_pos.loc[(df_all_pos.sent_len-df_all_pos.index)<=1,next_col1] = "end"
    
    return df_all_pos

In [9]:
df_all_pos1 = get_prev_next(df_all_pos1,"word")
df_all_pos1 = get_prev_next(df_all_pos1,"lemma")
df_all_pos1 = get_prev_next(df_all_pos1,"shape")
df_all_pos1 = get_prev_next(df_all_pos1,"pos_tag")
df_all_pos1 = get_prev_next(df_all_pos1,"word_type")

C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

C:\Users\user\Anaconda2\envs\py36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

In [10]:
df_all_pos1.columns

Index(['word', 'pos_tag', 'sid', 'sid1', 'word1', 'word_type', 'shape',
       'lemma', 'length', 'ind_num', 'sent_len', 'rel_position', 'prev_word',
       'word_next', 'prev_prev_word', 'word_next_next', 'prev_lemma',
       'lemma_next', 'prev_prev_lemma', 'lemma_next_next', 'prev_shape',
       'shape_next', 'prev_prev_shape', 'shape_next_next', 'prev_pos_tag',
       'pos_tag_next', 'prev_prev_pos_tag', 'pos_tag_next_next',
       'prev_word_type', 'word_type_next', 'prev_prev_word_type',
       'word_type_next_next'],
      dtype='object')

Step 4: categorizing string and number columns

In [11]:
num_cols = ['length','sent_len', 'rel_position']

str_cols = ['word', 'pos_tag', 'shape', 'lemma', 'word_type',
        'prev_word', 'word_next', 'prev_prev_word',
       'word_next_next', 'prev_lemma', 'lemma_next', 'prev_prev_lemma',
       'lemma_next_next', 'prev_shape', 'shape_next', 'prev_prev_shape',
       'shape_next_next', 'prev_pos_tag', 'pos_tag_next', 'prev_prev_pos_tag',
       'pos_tag_next_next','prev_word_type', 'word_type_next', 'prev_prev_word_type',
       'word_type_next_next']

x_test_num = df_all_pos1[num_cols]
x_test = df_all_pos1

Step 5: Loading the model files and tokenization files. The “x_test” dataset is suitably modified to input for the model run

In [12]:
import pickle
def pick_in(obj_name):
    fl_out1 = fl_out + "/" + obj_name
    pickle_in = open(fl_out1,"rb")
    mod1= pickle.load(pickle_in)
    
    return mod1

fl_out =  "model_punc"
pikl_list = ["tkn_list"]
for i in pikl_list:
    globals()[i] = pick_in(i)

from keras.models import load_model
final_model = load_model('model_punc/model_ner.h5')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


Converting the string dataset to tokenized inputs for embedding layers. Followed by concatenating the final string array with all the numeric data into a single dataset

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def conv_str_cols(col_te,num):

    col_te1 = tkn_list[num].texts_to_sequences(col_te)

    col_te2 = pad_sequences(col_te1, maxlen=1, dtype='int32', padding='pre')
    
    return col_te2

In [14]:
import numpy as np
for num,i in enumerate(str_cols):
    
    var1 = conv_str_cols(x_test[i],num)
    
    if(num==0):
        var_all_test = var1
    else:
        var_all_test = np.concatenate([var_all_test,var1],axis=1)

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Concatenate
from keras.layers.merge import concatenate
from keras.layers import Input, Dense, Dropout, Flatten
from keras.models import Model
from keras.utils import to_categorical
from keras.optimizers import Adam,SGD
from keras.layers import Embedding
from keras.layers import Reshape
from keras.layers import add

In [16]:
df_list_test=[]
for num,categoical_var in enumerate(range(var_all_test.shape[1]) ):
    df_list_test.append(var_all_test[:,num])
df_list_test.append(x_test_num)

Step 6: Apply the model and identify the predicted “organization” from the news corpus

In [17]:
pred1=final_model.predict(df_list_test)
pred = pred1.argmax(axis=-1)

In [18]:
pd.Series(pred).value_counts()

0    1941596
1     135599
dtype: int64

In [19]:
x_test["pred"] = pred

C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


we are concateating consecutive “organization” words in the same sentence to form one full word since our model is at word level, we are concatenating to form a chunk

In [20]:
x_test["pred"] = pred

list_set_words = []
sid_prev = 0
fg=0
sid1_list = []
for num,row in x_test[x_test.pred==1].iterrows():
    wrd = row["word"]
    ind_num = row["ind_num"]
    sid = row["sid"]
    sid1 = row["sid1"]
    
   
    if((num==0) or (sid_prev!=sid)):
        wrd_prev = wrd

    else:
        if((ind_num-ind_num_prev)==1):
            wrd_prev = wrd_prev + " " + wrd
            fg=1

        else:
            if(fg==1):
                list_set_words.append(wrd_prev)
                sid1_list.append(sid1_prev)
                fg=0
            wrd_prev = wrd
            
 
            
    ind_num_prev = ind_num
    sid_prev = sid
    sid1_prev = sid1
       
           


C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
df_all_ents = pd.DataFrame(sid1_list)
df_all_ents.columns = ["sid1"]
df_all_ents["ner_words"] = list_set_words

In [22]:
len(df_all_ents)

22762

In [23]:
df_all_ents.head()

,sid1,ner_words
0,1,Malaika Arora Khan
1,2,Indira Gandhi Institute of Medical
2,2,Dr Manish Mandal
3,2,Dr
4,2,All India Institute of Medical Sciences


Save this dataset to a file

In [29]:
df_all_ents.to_csv("only_ner_tagged.csv")